## Install `Reinforce` Package

In [ ]:
using Pkg

In [ ]:
Pkg.add("Reinforce")

## Create Multi-armed Bandit Environment

In [ ]:
using Reinforce

In [ ]:
MultiArmedBandit(10, 1000)  # 10 arms, 1000 iterations

In [ ]:
envs = [MultiArmedBandit(10, 1000) for _ ∈ 1:2000];  # generate 2000 k-armds bandit problems

## Define your own Policy

In [ ]:
struct ϵGreedy <: AbstractPolicy
    Q::Dict{Int,Float64}
    n::Dict{Int,Float64}
    ϵ::Float64
end
ϵGreedy(; ϵ = .1) = ϵGreedy(Dict{Int,Float64}(), Dict{Int,Float64}(), ϵ)

function Reinforce.action(π::ϵGreedy, r, s, A)
    (rand() < π.ϵ) && return rand(A)

    qs = [get(π.Q, a, 0) for a ∈ A]
    A[argmax(qs)]
end

In [ ]:
function run_single(env; ϵ)
    p = ϵGreedy(ϵ = ϵ)
    r_seq = Float64[]
    run_episode(env, p) do (s, a, r, s′)
        n′ = get(p.n, a, 0) + 1
        q = get(p.Q, a, 0)
        q′ = q + (1 / n′) * (r - q)
        # update
        p.n[a] = n′
        p.Q[a] = q′
        # save history
        push!(r_seq, r)
    end

    r_seq
end

In [ ]:
R₁ = mapreduce(e -> run_single(e, ϵ = 0.1), hcat, envs)

In [ ]:
R₂ = mapreduce(e -> run_single(e, ϵ = 0.01), hcat, envs)

In [ ]:
using Statistics

In [ ]:
mean(R, dims = 2)

## Plot the performance

In [ ]:
Pkg.add("Plots")

In [ ]:
using Plots

In [ ]:
plot(mean(R₁, dims = 2), label = "epsilon = 0.1", ylabel = "average reward", xlabel = "steps")
plot!(mean(R₂, dims = 2), label = "epsilon = 0.01")

# Homework

##  Simple bandit algorithm

1. Please implement the $\epsilon$-greedy algorithm with a constant step-size $\alpha$, as described in the equation (2.5).
2. Run 2000 problems with 1000 steps in each.
   Compare the following $\epsilon$ settings:
  - $\epsilon = 0.01$
  - $\epsilon = 0.1$
  - $\epsilon = 0.2$
 
 Please plot the result.
3. Run 2000 problems with 5000 steps in each and plot the result.

4. Adjust the standard diviation of `MultiArmedBandit`
```
MultiArmedBandit(10, 1000, σ = 5)
```
  Then, re-run the setting described in 2. and 3.
  
5. Compare the results between low and high variance. That is,
   $\sigma = 1$ vs $\sigma = 5$. Describe your finding.

## Optimistic Inital Values

1. Please implmenet the algorithm **greedy with optimistic initial values** described in page 34.
2. Plot the result of different initial values:
  - $Q_1 = 1$
  - $Q_1 = 5$

## Upper-Confidence-Bound Action Selection

1. Please implement the UCB algorithm describe in equation (2.10) page 35.
   Consider following setting of $c$:
   - $c = 1$
   - $c = 2$
   - $c = 3$